In [73]:
import kfp
import kfp.components as components
import kfp.dsl as dsl
from kfp.components import InputPath, OutputPath

In [74]:
def train_export_model(trainingjobName: str, epochs: str, version: str):
    
    import tensorflow as tf
    from numpy import array
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import Flatten, Dropout, Activation
    from tensorflow.keras.layers import LSTM
    import numpy as np
    print("numpy version")
    print(np.__version__)
    import pandas as pd
    import os
    from featurestoresdk.feature_store_sdk import FeatureStoreSdk
    from modelmetricsdk.model_metrics_sdk import ModelMetricsSdk
    
    fs_sdk = FeatureStoreSdk()
    mm_sdk = ModelMetricsSdk()
    
    features = fs_sdk.get_features(trainingjobName, ['DRB.UEThpDl','RRU.PrbUsedDl','RRC.ConnMean','QosFlow.TotPdcpPduVolumeDl', 'PEE.AvgPower', 'timestamp'])
    
    print("all feature:")
    print(features.columns)
    print(features.head())
    
    
    name_file="https://drive.usercontent.google.com/download?id=1V4QM4hrY2n9IBgMrUqCASVT8YrBZruQb&export=download&authuser=0&confirm=t&uuid=1bd3af69-a9df-444d-b907-f25be9f30593&at=APZUnTW02bjx9IwiYRtRjdcVzw-j:1722398536728"
    import urllib.request
    urllib.request.urlretrieve(
            name_file, 'model_tensor.hdf5')
    print("url berhasil")
    model = tf.keras.models.load_model('./model_tensor.hdf5')
    print("modelnya adalah: ")
    print(model)
    #xx = y
    #yy = yhat
    model.save("./")
    import json
    data = {}
    data['metrics'] = [0]
    #data['metrics'].append({'Accuracy': str(np.mean(np.absolute(np.asarray(xx)-np.asarray(yy))<5))})
    
    mm_sdk.upload_metrics(data, trainingjobName, version)
    mm_sdk.upload_model("./", trainingjobName, version)

In [75]:
BASE_IMAGE = "traininghost/pipelineimage:latest"

In [76]:
def train_and_export(trainingjobName: str, epochs: str, version: str):
    trainOp = components.func_to_container_op(train_export_model, base_image=BASE_IMAGE)(trainingjobName, epochs,version)
    # Below line to disable caching of pipeline step
    trainOp.execution_options.caching_strategy.max_cache_staleness = "P0D"
    trainOp.container.set_image_pull_policy("IfNotPresent")

In [77]:
@dsl.pipeline(
    name="qoe Pipeline",
    description="qoe",
)
def super_model_pipeline( 
    trainingjob_name: str, epochs: str, version: str):
    
    train_and_export(trainingjob_name, epochs, version)

In [78]:
pipeline_func = super_model_pipeline
file_name = "qoe_model_pipeline"

kfp.compiler.Compiler().compile(pipeline_func,  
  '{}.zip'.format(file_name))

In [79]:
import requests
pipeline_name="qoe_model_pipeline"
pipeline_file = file_name+'.zip'
requests.post("http://tm.traininghost:32002/pipelines/{}/upload".format(pipeline_name), files={'file':open(pipeline_file,'rb')})

<Response [200]>